In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import h5py
import scipy.signal
from tqdm import tqdm

In [ ]:
CNN_OUT_FILE =  "/media/fibe/FIO_2018/synth/synth_cnn.nc" 
PU_OUT_FILE =  "/media/fibe/FIO_2018/synth/synth_pu.nc" 
GT_FILE = "/media/fibe/FIO_2018/synth/synth_sequence.h5"



plt.rcParams.update({'font.size': 18})

In [ ]:
def crossings_nonzero_pos2neg(data):
    pos = data > 0
    return (pos[:-1] & ~pos[1:]).nonzero()[0]

def extract_timeserie( data, t ):
    print(" - Computing statistics on the grid center timeserie")
    timeserie = data[:,halfgridsize_i,halfgridsize_j] * 1E-3
    timeserie = np.array( timeserie - np.mean(timeserie) )
    dt = t[2]-t[1]
    print("   dt: ",dt)


    crossings = crossings_nonzero_pos2neg(timeserie)

    dmins = []
    dmaxs = []
    for ii in range( np.size(crossings)-1 ):
        datarange = np.arange(crossings[ii], crossings[ii+1])
        data = timeserie[ datarange ]
        dmax = np.argmax(data)
        dmin = np.argmin(data)
        dmins.append( datarange[dmin] )
        dmaxs.append( datarange[dmax] )
        
    waveheights = np.array(timeserie[dmaxs]) - np.array(timeserie[dmins])
    q = np.quantile( waveheights, 2.0/3.0)
    print("   timeserie quantile: ", q)

    highestthirdwaves = waveheights[ waveheights>q ]
    H13 = np.mean(highestthirdwaves)
    print("   H1/3: ", H13)
    return timeserie, crossings, dmins, dmaxs, H13

#plt.savefig(os.path.join(self.scriptpath,"timeserie.png"))


In [ ]:
root_cnn = Dataset( CNN_OUT_FILE )
root_pu = Dataset( PU_OUT_FILE )

print( root_cnn["/Z"].shape )
print( root_pu["/Z"].shape )

sequence_start = 16
sequence_end = np.min([root_cnn["/Z"].shape[0], root_pu["/Z"].shape[0]])-1

Zcnn = root_cnn["/Z"][sequence_start:sequence_end,...]
Zpu = root_pu["/Z"][sequence_start:sequence_end,...]

f = h5py.File( GT_FILE, "r" )
Zgt = f["/0000/data"][sequence_start:sequence_end,:,:]*1000.0

nsamples = root_cnn["/Z"].shape[0]
gridsize = root_cnn["/Z"].shape[1:3]

halfgridsize_i = gridsize[0]//2
halfgridsize_j = gridsize[1]//2

valid_i_start = gridsize[0]//3-35
valid_i_end = gridsize[0]//3+36
valid_j_start = halfgridsize_j-35
valid_j_end = halfgridsize_j+36

In [ ]:
IDX = 80
mask = np.isnan( Zcnn[IDX,:,:])

plt.figure(figsize=(30,10))
plt.subplot(1,3,1)
Z = Zgt[IDX,:,:]

zmin = np.amin(Z)
zmax = np.amax(Z)

Z[mask] = np.nan
plt.imshow(Z, vmin=zmin, vmax=zmax)

plt.subplot(1,3,2)
Z = Zcnn[IDX,:,:]
plt.imshow(Z, vmin=zmin, vmax=zmax)
plt.subplot(1,3,3)
Z = Zpu[IDX,:,:]
Z[mask] = np.nan
plt.imshow(Z, vmin=zmin, vmax=zmax)

plt.figure(figsize=(30,10))
plt.subplot(1,3,1)
Z = Zgt[IDX,valid_i_start:valid_i_end, valid_j_start:valid_j_end]
print("GT Hs: ", 4*np.std(Z) )
print("GT :", np.mean( np.abs(Z-np.mean(Z)) ))
plt.imshow(Z, vmin=zmin, vmax=zmax)
plt.subplot(1,3,2)
Z = Zcnn[IDX,valid_i_start:valid_i_end, valid_j_start:valid_j_end]
print("CNN Hs: ", 4*np.std(Z) )
print("CNN :", np.mean( np.abs(Z-np.mean(Z)) ))
plt.imshow(Z, vmin=zmin, vmax=zmax)
plt.subplot(1,3,3)
Z = Zpu[IDX,valid_i_start:valid_i_end, valid_j_start:valid_j_end]
print("Pu Hs: ", 4*np.std(Z) )
print("Pu :", np.mean( np.abs(Z-np.mean(Z)) ))
plt.imshow(Z,vmin=zmin, vmax=zmax)

In [ ]:

t = root_cnn["/time"][sequence_start:sequence_end]

plt.figure( figsize=(30,10))

ST=100
ED=400

dt = t[2]-t[1]

timeserie_gt, crossings, dmins, dmaxs, H13_gt = extract_timeserie( Zgt, t )
plt.plot(t[ST:ED], timeserie_gt[ST:ED], "k", linewidth=5 )

timeserie_cnn, crossings, dmins, dmaxs, H13_cnn = extract_timeserie( Zcnn, t )
plt.plot(t[ST:ED], timeserie_cnn[ST:ED], "b", linewidth=3 )

timeserie_pu, crossings, dmins, dmaxs, H13_pu = extract_timeserie( Zpu, t )
plt.plot(t[ST:ED], timeserie_pu[ST:ED], "r", linewidth=3 )

plt.grid()
plt.legend([ "GT  (H1/3 = %1.3f)"%H13_gt,
             "CNN (H1/3 = %1.3f)"%H13_cnn,
             "PU  (H1/3 = %1.3f)"%H13_pu,
 ])
plt.title("Surface Elevation at Grid Center")
plt.xlabel("Time (secs.)")
plt.ylabel("Height (m)")


print("Pearson correlation (CNN vs GT)")
print( np.corrcoef(timeserie_gt, timeserie_cnn) )
print("Pearson correlation (PU vs GT)")
print( np.corrcoef(timeserie_gt, timeserie_pu) )

In [ ]:
Zcube_gt = np.array( Zgt[:,valid_i_start:valid_i_end, valid_j_start:valid_j_end] * 1E-3 )
Zcube_gt = Zcube_gt - np.mean(Zcube_gt)
Hs = 4.0*np.std( Zcube_gt )
print("GT   Hs: ", Hs)

Zcube_cnn = np.array( Zcnn[:,valid_i_start:valid_i_end, valid_j_start:valid_j_end] * 1E-3 )
Zcube_cnn = Zcube_cnn - np.mean(Zcube_cnn)
Hs = 4.0*np.std( Zcube_cnn) 
print("CNN  Hs: ", Hs)


Zcube_pu = np.array( Zpu[:,valid_i_start:valid_i_end, valid_j_start:valid_j_end] * 1E-3 )
Zcube_pu = Zcube_pu - np.mean(Zcube_pu)
Hs = 4.0*np.std( Zcube_pu )
print("PU  Hs: ", Hs)

In [ ]:
Hs_gt = 4.0*np.std( Zcube_gt, axis=(1,2) )
Hs_cnn = 4.0*np.std( Zcube_cnn, axis=(1,2)) 
Hs_pu = 4.0*np.std( Zcube_pu, axis=(1,2) )

plt.figure( figsize=(20,10))
#plt.plot( Hs_gt, 'k')
Hs_error_pu = np.abs( Hs_pu-Hs_gt )
Hs_error_cnn = np.abs( Hs_cnn-Hs_gt )
plt.plot( Hs_error_pu[:100], 'r')
plt.plot( Hs_error_cnn[:100], 'b')
plt.grid()
plt.legend(["PU","CNN"])

In [ ]:
MSE_pu = np.mean( np.square(Zcube_gt - Zcube_pu) )
print("MSE pu: ", MSE_pu)
MSE_cnn = np.mean( np.square(Zcube_gt - Zcube_cnn) )
print("MSE cnn: ", MSE_cnn)

MSE_pu = np.mean( np.square(Zcube_gt - Zcube_pu), axis=(1,2) )
print("MSE pu: ", MSE_pu)
MSE_cnn = np.mean( np.square(Zcube_gt - Zcube_cnn), axis=(1,2) )
print("MSE cnn: ", MSE_cnn)

plt.figure( figsize=(20,10))
plt.plot( MSE_pu[:100], 'r')
plt.plot( MSE_cnn[:100], 'b')
plt.grid()
plt.legend(["PU","CNN"])

In [ ]:

def compute_omnispec( timeserie, data, dt, valid_samples_i, valid_samples_j ):
    NPSEG = 300
    f, S = scipy.signal.csd(timeserie, timeserie, 1.0/dt, nperseg=NPSEG )

    for ii in tqdm(valid_samples_i):
        for jj in valid_samples_j:
            timeserie_neigh = data[:,ii,jj] * 1E-3
            timeserie_neigh = timeserie_neigh - np.mean(timeserie_neigh)
            _, S_neig = scipy.signal.csd(timeserie_neigh, timeserie_neigh, 1.0/dt, nperseg=NPSEG )
            S += S_neig

    S = S / float( np.size(valid_samples_i)*np.size(valid_samples_j) + 1)
    return f,S

f,Sgt = compute_omnispec( timeserie_gt, Zgt, dt, np.arange(valid_i_start,valid_i_end), np.arange(valid_j_start, valid_j_end) )
f,Scnn = compute_omnispec( timeserie_cnn, Zcnn, dt, np.arange(valid_i_start,valid_i_end), np.arange(valid_j_start, valid_j_end) )
f,Spu = compute_omnispec( timeserie_pu, Zpu, dt, np.arange(valid_i_start,valid_i_end), np.arange(valid_j_start, valid_j_end) )

topfreq = np.where(f>0.7)[0][0]

plt.figure( figsize=(10,10) )
plt.loglog( f[:topfreq], Sgt[:topfreq], "k")
plt.loglog( f[:topfreq], Scnn[:topfreq], "b")
plt.loglog( f[:topfreq], Spu[:topfreq], "r")
#plt.xticks([1E-2,1E-1,1E0])
plt.grid(which='minor')
plt.legend(["GT","CNN", "PU"])
plt.ylabel("S (m^2s)")
plt.xlabel("f_a (1/s)")
plt.title("Spectrum (Welch method) averaged in central grid region")

# Compute Hs
dFreq = np.gradient( f )
m0 = np.sum( Sgt*dFreq )
m1 = np.sum( f*Sgt*dFreq )
Hm0 = 4.0 * np.sqrt( m0 )
print("GT   Hm0: ", Hm0)
m0 = np.sum( Scnn*dFreq )
m1 = np.sum( f*Scnn*dFreq )
Hm0 = 4.0 * np.sqrt( m0 )
print("CNN   Hm0: ", Hm0)
m0 = np.sum( Spu*dFreq )
m1 = np.sum( f*Spu*dFreq )
Hm0 = 4.0 * np.sqrt( m0 )
print(" PU   Hm1: ", Hm0)

# Peak frequency
pp = f[np.argmax( Sgt )]
print("   Peak frequency (Hz): ", pp)

# Average Period Tm01
Tm01 = m0/m1
print("   Tm01: ", Tm01)

In [ ]:
np.where(f>1.0)[0][0]

In [ ]:
Z_to_process = Zcnn

print(" - Analyzing space-time 3D spectrum")
Z = Z_to_process[3,:,:]
N = Z.shape[0]
Nm = int( N/2 )
dy = (root_cnn["/Y_grid"][2,0] - root_cnn["/Y_grid"][1,0])/1000.0
dx = (root_cnn["/X_grid"][0,2] - root_cnn["/X_grid"][0,1])/1000.0
print("   grid dx,dy: ", dx,dy)

if np.abs( dx-dy ) < 1E-3:
    dy = dx  # force dx = dy if very close to avoid numerical errors

# Extract a central part of the Zcube
N = 120
min_freq = 0.25
max_freq = 0.7
num_plots = 10
segments = 5 

sequence_length = np.size(timeserie_gt)
Nt = int(sequence_length / segments)
if Nt%2 > 0:
    Nt+=1
seg_shift = int(Nt/2)

Zcube_mr = int( Z_to_process.shape[1] / 2 )
Zcube_mc = int( Z_to_process.shape[2] / 2 )
r_start, r_end = Zcube_mr-int(N/2)-20, Zcube_mr+int(N/2)-20+1
c_start, c_end = Zcube_mc-int(N/2), Zcube_mc+int(N/2)+1 

Nx = r_end - r_start
Ny = c_end - c_start
print("   Nx,Ny,Nt: ",Nx,Ny,Nt)

kx_max=(2.0*np.pi/dx)/2.0
ky_max=(2.0*np.pi/dy)/2.0
f_max= (1.0/dt)/2.0
dkx=2.0*np.pi/(dx*np.floor(Nx/2.0)*2.0)
dky=2.0*np.pi/(dy*np.floor(Ny/2.0)*2.0)
df =1.0/(dt*np.floor(Nt/2.0)*2.0)

#print("   kx_max, ky_max, f_max, dkx, dky, df: ", kx_max, ky_max, f_max, dkx, dky, df)

assert( Nx%2 != 0)
assert( Ny%2 != 0)
assert( Nt%2 == 0)

kx=np.arange(-kx_max,kx_max+dkx,dkx)
ky=np.arange(-ky_max,ky_max+dky,dky)

if Nt%2==0:
    f=np.arange(-f_max, f_max, df)
else:
    f=np.arange(-f_max, f_max+df, df)

KX, KY = np.meshgrid( kx, ky )
dkx=kx[3]-kx[2]
dky=ky[3]-ky[2]
KXY=np.sqrt(KX**2+KY**2)
print("   kdx, dky: ", dkx, dky)


hanningx = scipy.signal.windows.hann(KX.shape[0])
hanningy = scipy.signal.windows.hann(KX.shape[1])
hanningt = scipy.signal.windows.hann(Nt)

Win3Dhann = np.tile( np.expand_dims( hanningx, axis=-1) * hanningy, (Nt,1,1) ) *  np.tile( np.expand_dims( np.expand_dims( hanningt, axis=-1 ), axis=-1 ), (1, KX.shape[0], KX.shape[1]) )
assert( KX.shape == Win3Dhann.shape[1:] )

#  window correction factors
wc2x = 1.0/np.mean(hanningx**2)
wc2y = 1.0/np.mean(hanningy**2)
wc2t = 1.0/np.mean(hanningt**2)
wc2xy  = wc2x *wc2y
wc2xyt = wc2xy*wc2t

print("   dt, dkx, dky, df: ",dt, dkx, dky, df)

# Fix for rounding errors
r_end = r_start + Win3Dhann.shape[1]
c_end = c_start + Win3Dhann.shape[2]

S_welch = np.zeros_like( Win3Dhann )
n_samples = 0
print("   Computing 3D FFT via Welch's method... ", end="")
for ii in tqdm(range(segments*2)):
    #print("Welch sample %d/%d"%(ii+1,segments*2))
    Zcube_small = np.array( Z_to_process[(ii*seg_shift):(ii*seg_shift+Nt), r_start:r_end, c_start:c_end ] )
    if Zcube_small.shape[0] != Nt:
        break
        
    Zcube_w = (Zcube_small - np.mean(Zcube_small) ) * Win3Dhann
    
    S = np.fft.fftshift( np.fft.fftn( Zcube_w, norm="ortho" ) )
    S /= (S.shape[0]*S.shape[1]*S.shape[2])
    S = np.abs(S)**2 / (dkx*dky*df)
    #-----------------------------
    #%%%%% corrects for window
    #----------------------------
    #%% FABIEN
    S *= wc2xyt
    
    # Store
    S_welch += S    
    n_samples += 1
    
S_welch /= n_samples    
print(" Done!")



In [ ]:

min_freq = 0.35
max_freq = 0.7
num_plots = 10

start_freq_ii = np.argmin( np.abs(f-min_freq) )
end_freq_ii = np.argmin( np.abs(f-max_freq) )
indices = np.round( np.linspace(start_freq_ii, end_freq_ii, num_plots ) ).astype(np.uint32)

print("  Generating plots")
kk=0
for ii in tqdm(indices):

    plt.figure( figsize=(11,10))    

    #dummy = np.flipud( 2* np.mean(S_welch[ mdt+ii-1:mdt+ii+2,:,:], axis=0) )    
    dummy = 2* np.mean(S_welch[ ii-1:ii+2,:,:], axis=0) 
    
    dummy_cen = np.copy(dummy)
    dummy_cen[ int(dummy_cen.shape[0]/2)-1:int(dummy_cen.shape[0]/2)+1, int(dummy_cen.shape[1]/2)-1:int(dummy_cen.shape[1]/2)+1 ] = 0
    maxidx = np.unravel_index( np.argmax(dummy_cen), dummy_cen.shape )
    
    qp=( np.arctan2( KY[ maxidx[0],maxidx[1] ], KX[ maxidx[0],maxidx[1] ]) )/np.pi*180.0
    if qp<0:
        qp=qp+360

    kp=np.sqrt( KX[ maxidx[0],maxidx[1] ]**2 + KY[ maxidx[0],maxidx[1] ]**2 )

    plt.pcolor(KX,KY, 10*np.log10(dummy) )
    plt.clim( 10*np.array([-5.0 + np.amax(np.log10(dummy)), -0+np.amax(np.log10(dummy))]) )
    plt.colorbar()

    #plt.scatter( [KX[ maxidx[0],maxidx[1] ]], [KY[ maxidx[0],maxidx[1] ]], marker="x", s=100, c="k" )

    plt.ylim([-3.0,3.0])
    plt.xlim([-3.0,3.0])

    plt.xlabel("Kx (rad/m)")
    plt.ylabel("Ky (rad/m)")
    plt.title("S_kx_ky, fa=%3.2f (Hz).\n Peak angle: %3.0f°, mag: %2.3f (rad/m)\n"%( f[ii],qp,kp ) )
    kk+=1